In [1]:
import os
import re

# Data preprocessing

Extract experiment settings and No. of violations

## get data with default settings

In [2]:
def parseDefaultOneLine(line, shapeCost=8, markerCost=32) -> list:
    """Parse 1 line of iteration setting"""
    # remove delimiters
    tmp = re.split(r'[\{\}\,]+', line)
    iteData = []
    # get size, offset, and mazeEndIter
    iteData.append(int(re.sub(r'\s+', '', tmp[1])))
    iteData.append(int(re.sub(r'\s+', '', tmp[2])))
    iteData.append(int(re.sub(r'\s+', '', tmp[3])))
    # get 1st cost
    compo = re.split(r'[\s\*]+', tmp[4])
    if len(compo) == 2: # no coefficient before shapeCost
        iteData.append(shapeCost)
    else:               # shapeCost with coefficient
        iteData.append(int(compo[1]) * shapeCost)
    # get 2nd cost
    compo = re.split(r'[\s\*]+', tmp[5])
    if len(compo) == 2: # no coefficient before shapeCost or markerCost
        if compo[1] == 'shapeCost':
            iteData.append(shapeCost)
        elif compo[1] == '0':
            iteData.append(0)
        else:
            iteData.append(markerCost)
    else:               # cost with coefficient
        iteData.append(int(compo[1]) * markerCost)
    # get 3rd cost
    compo = re.split(r'[\s\*]+', tmp[6])
    if len(compo) == 2: # no coefficient before shapeCost
        iteData.append(shapeCost)
    else:               # shapeCost with coefficient
        iteData.append(int(compo[1]) * shapeCost)
    # get decay
    iteData.append(float(re.sub(r'\s+', '', tmp[7])))
    # get mode
    mode = re.sub(r'[\s\:]+', '', tmp[8])
    if mode[-7:] == 'ModeALL':
        iteData.append('ALL')
    elif mode[-7:] == 'ModeDRC':
        iteData.append('DRC')
    else:
        iteData.append('NEARDRC')
    # get followGuide
    followGuide = re.sub(r'\s+', '', tmp[9])
    if followGuide == 'false':
        iteData.append(False)
    else:
        iteData.append(True)
        
    return iteData

In [3]:
# parse default strategy
def parseDefaultStrategy(inPath, outPath, shapeCost=8, markerCost=32) -> list:
    """Parse default strategy, save result to file, and return parsing status.
    
    Inputs:
    inPath: input FlexDR.cpp file path
    outPath: output result file path
    shapeCost: default shapeCost
    markerCost: default markerCost
    
    Return:
    Return list of strategy data.
    """
    # check file exists
    inputPath = os.path.join(inPath, "FlexDR.cpp")
    if not os.path.exists(inputPath):
        print("FlexDR.cpp for default setting doesn't exist in {}!".format(inputPath))
        return None
    # open, read, and extract coefficients from input file
    data = [] # temp coeff data
    count = 1
    with open(inputPath, 'r') as f:
        line = f.readline()
        while count < 1030:
            count = count + 1
            # check strategy() function
            if line == "static std::vector<FlexDR::SearchRepairArgs> strategy()\n":
                print("Start parsing iteration data!")
                # skip prefix lines
                line = f.readline()
                line = f.readline()
                line = f.readline()
                line = f.readline()
                count = count + 3
                # parse iteration settings
                while line != "  };\n":
                    # parse data
                    tempData = parseDefaultOneLine(line, shapeCost=shapeCost, markerCost=markerCost)
                    data.append(tempData)
                    # read next line
                    line = f.readline()
                    count = count + 1
                print("End parsing iteration data!")
            else:
                line = f.readline()
    # check output path exists
    if not os.path.exists(outPath):
        print("Output path {} for default setting data doesn't exist!".format(outPath))
        return None
    # create and write to output file
    with open(os.path.join(outPath, "defaultStrategy.txt"), 'w') as f:
        for line in data:
            for element in line:
                f.write("{},".format(element))
            f.write("@")
    return data

In [4]:
inPath = "/home/jborg/Data/Research/DLPnR/DL-PnR/data/default"
outPath = "/home/jborg/Data/Research/DLPnR/DL-PnR/src"
data = parseDefaultStrategy(inPath, outPath)

Start parsing iteration data!
End parsing iteration data!


In [5]:
def parseJSONFile(filename) -> list:
    """parse JSON file and return an output vector
    
    Inputs: 
    filename: path to 5_3_route.json file
    
    Return:
    A output vector containing the number of violations for each iteration
    """
    # check file exists
    if not os.path.exists(filename):
        print(filename)
        print("JSON file {} doesn't exist!".format(filename))
        return None
    # read and parse lines
    data = []
    with open(filename, 'r') as f:
        while True:
            line = f.readline()
            tmp = re.split(r'[\"\,\s\:]+', line)
            if len(tmp) > 2 and tmp[1] == 'detailedroute__route__drc_errors__iter':
                data.append(int(tmp[3]))
            if line == "}":
                break
    # return result
    return data

In [6]:
# parse default log files
def parseDefaultLogFiles(inPath, outPath) -> int:
    """Parse log files in the whole directory
    
    Inputs:
    inPath: log file directory
    outPath: output file path
    
    Return:
    Return list of output data.
    """
    # check path exists
    if not os.path.exists(inPath):
        print("Default log file path {} doesn't exist!".format(inPath))
        return None
    # check 5_3_route.json file exists and get outputs
    data = []
    for root, dirs, files in os.walk(inPath):
        for file in files:
            if file == "5_3_route.json":
                # parse json file for outputs
                data.append(parseJSONFile(os.path.join(root, file)))
                print("Data of {}: {}".format(os.path.join(root, file), parseJSONFile(os.path.join(root, file))))
    # check output path exists
    if not os.path.exists(outPath):
        print("Output path {} for default log data doesn't exist!".format(outPath))
        return None
    # create and write to output file
    with open(os.path.join(outPath, "defaultOutputs.txt"), 'w') as f:
        for line in data:
            for element in line:
                f.write("{},".format(element))
            f.write("\n")
            
    return data

In [7]:
data = parseDefaultLogFiles("/home/jborg/Data/Research/DLPnR/DL-PnR/data/default", 
                     "/home/jborg/Data/Research/DLPnR/DL-PnR/src")
data

Data of /home/jborg/Data/Research/DLPnR/DL-PnR/data/default/logs/gf180/riscv32i/base/5_3_route.json: [10178, 569, 499, 1, 0]
Data of /home/jborg/Data/Research/DLPnR/DL-PnR/data/default/logs/gf180/uart-blocks/base/5_3_route.json: [226, 18, 9, 0]
Data of /home/jborg/Data/Research/DLPnR/DL-PnR/data/default/logs/gf180/aes/base/5_3_route.json: [24561, 1453, 1030, 2, 0]
Data of /home/jborg/Data/Research/DLPnR/DL-PnR/data/default/logs/gf180/ibex/base/5_3_route.json: [16946, 864, 610, 8, 0]
Data of /home/jborg/Data/Research/DLPnR/DL-PnR/data/default/logs/gf180/jpeg/base/5_3_route.json: [20369, 983, 530, 0]
Data of /home/jborg/Data/Research/DLPnR/DL-PnR/data/default/logs/gf180/uart-blocks_uart_rx/base/5_3_route.json: [205, 10, 7, 0]
Data of /home/jborg/Data/Research/DLPnR/DL-PnR/data/default/logs/asap7/mock-array_Element/base/5_3_route.json: [155, 37, 40, 0]
Data of /home/jborg/Data/Research/DLPnR/DL-PnR/data/default/logs/asap7/mock-array/base/5_3_route.json: [1103, 302, 80, 0]
Data of /home/jb

[[10178, 569, 499, 1, 0],
 [226, 18, 9, 0],
 [24561, 1453, 1030, 2, 0],
 [16946, 864, 610, 8, 0],
 [20369, 983, 530, 0],
 [205, 10, 7, 0],
 [155, 37, 40, 0],
 [1103, 302, 80, 0],
 [32846, 1962, 1482, 1, 0],
 [6981, 684, 613, 3, 0],
 [88718, 11103, 7910, 350, 43, 4, 0],
 [12295, 1090, 911, 18, 0],
 [232, 16, 12, 1, 0],
 [55009,
  47472,
  51514,
  7698,
  953,
  179,
  106,
  10,
  10,
  11,
  7,
  6,
  5,
  5,
  4,
  4,
  4,
  1059,
  4,
  3,
  3,
  3,
  3,
  3,
  3,
  5,
  5,
  5,
  5,
  4,
  4,
  4,
  4,
  3,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  3,
  3,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2],
 [177521,
  168983,
  142289,
  54869,
  10385,
  1540,
  790,
  505,
  72,
  60,
  32,
  24,
  19,
  18,
  13,
  11,
  10,
  11,
  5,
  6,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0],
 [117297, 97675, 94141, 22881, 2915, 407, 69, 35, 24, 21, 10, 10, 7, 1, 1, 0],
 [325, 329, 157, 15, 0],
 [928, 1378, 1305, 285, 3, 0],
 [11941, 13

## get data with automatically generated settings

In [8]:
def parseOneLine(line) -> list:
    """Parse 1 line of iteration setting"""
    # remove delimiters
    tmp = re.split(r'[\{\}\,\t]+', line)
    iteData = []
    # get size, offset, mazeEndIter, 3 types of costs, and decay
    iteData.append(int(re.sub(r'\s+', '', tmp[1])))
    iteData.append(int(re.sub(r'\s+', '', tmp[2])))
    iteData.append(int(re.sub(r'\s+', '', tmp[3])))
    iteData.append(int(re.sub(r'\s+', '', tmp[4])))
    iteData.append(int(re.sub(r'\s+', '', tmp[5])))
    iteData.append(float(re.sub(r'\s+', '', tmp[6])))
    # get mode
    mode = re.sub(r'[\s\:]+', '', tmp[7])
    if mode[-3:] == 'ALL':
        iteData.append('ALL')
    elif mode[-7:] == 'NEARDRC':
        iteData.append('NEARDRC')
    else:
        iteData.append('DRC')
    # get followGuide
    followGuide = re.sub(r'\s+', '', tmp[8])
    if followGuide == 'false':
        iteData.append(False)
    else:
        iteData.append(True)
        
    return iteData

In [9]:
# line = "			{7,	0,	3,	7,	71,	0.745245,	RipUpMode::ALL,	true},"
# parseOneLine(line)

In [10]:
# parse default strategy
def parseStrategy(inPath) -> list:
    """Parse default strategy, save result to file, and return parsing status.
    
    Inputs:
    inPath: input FlexDR.cpp file path
    shapeCost: default shapeCost
    markerCost: default markerCost
    
    Return:
    Return list of strategy data.
    """
    # check file exists
    inputPath = os.path.join(inPath, "FlexDR.cpp")
    if not os.path.exists(inputPath):
        print("FlexDR.cpp for default setting doesn't exist in {}!".format(inputPath))
        return None
    # open, read, and extract coefficients from input file
    data = [] # temp coeff data
    count = 1
    with open(inputPath, 'r') as f:
        line = f.readline()
        while count < 1030:
            count = count + 1
            # check strategy() function
            if line == "static std::vector<FlexDR::SearchRepairArgs> strategy()\n":
                print("Start parsing iteration data!")
                # skip prefix lines
                line = f.readline()
                line = f.readline()
                line = f.readline()
                line = f.readline()
                count = count + 3
                # parse iteration settings
                while line != "  };\n":
                    # parse data
                    tempData = parseOneLine(line)
                    data.append(tempData)
                    # read next line
                    line = f.readline()
                    count = count + 1
                print("End parsing iteration data!")
            else:
                line = f.readline()

    return data

In [11]:
# parseStrategy("/home/jborg/Data/Research/DLPnR/DL-PnR/data/random/logs/asap7/aes")

In [12]:
def parseStrategyAndLog(inPath, outPath) -> int:
    """Parse artificially generated strategy, save result to file, and return parsing status.
    
    Inputs:
    inPath: input FlexDR.cpp file path
    outPath: output result file path
    
    Return:
    Return 0 for correct result. 1 for Error
    """
    # check input file exists
    if not os.path.exists(inPath):
        print("Log file path {} doesn't exist!".format(inPath))
        return 1
    # traverse design folders
    coeffData = []
    outData = []
    for Folder in os.listdir(inPath):
        for folder in os.listdir(os.path.join(inPath, Folder)):
            # check FlexDR.cpp and 5_3_route.json file exists
            currentDir = os.path.join(inPath, Folder, folder)
            print("\n\n")
            print(currentDir)
            cppFilename = os.path.join(currentDir, "FlexDR.cpp")
            jsonFilename = os.path.join(currentDir, "base/5_3_route.json")
            if os.path.exists(cppFilename) and os.path.exists(jsonFilename):
                print("\tGet data from {}".format(currentDir))
                # get iteration settings
                strategy = parseStrategy(currentDir)
                coeffData.append(strategy)
                print(coeffData)
                # get outputs
                out = parseJSONFile(jsonFilename)
                outData.append(out)
                print(out)
    # check output path exists
    if not os.path.exists(outPath):
        print("Output path {} doesn't exist!".format(outPath))
        return 1
    # create and write to output file
    with open(os.path.join(outPath, "strategy.txt"), 'w') as f:
        for strategy in coeffData:
            for line in strategy:
                for element in line:
                    f.write("{},".format(element))
                f.write("@")
            f.write("\n")
    with open(os.path.join(outPath, "outputs.txt"), 'w') as f:
        for line in outData:
            for element in line:
                f.write("{},".format(element))
            f.write("\n")
    return 0

In [13]:
inPath = "/home/jborg/Data/Research/DLPnR/DL-PnR/data/random/logs"
outPath = "/home/jborg/Data/Research/DLPnR/DL-PnR/src"
parseStrategyAndLog(inPath, outPath)




/home/jborg/Data/Research/DLPnR/DL-PnR/data/random/logs/gf180/riscv32i
	Get data from /home/jborg/Data/Research/DLPnR/DL-PnR/data/random/logs/gf180/riscv32i
Start parsing iteration data!
End parsing iteration data!
[[[7, 0, 63, 7, 91, 0.897397, 'ALL', True], [7, -1, 9, 12, 8, 0.563063, 'ALL', True], [7, -2, 3, 4, 69, 0.951451, 'ALL', True], [7, -3, 51, 6, 94, 0.611111, 'DRC', False], [7, -4, 51, 4, 4, 0.876376, 'DRC', False], [7, -5, 20, 8, 73, 0.654154, 'DRC', False], [7, -6, 3, 13, 75, 0.95045, 'DRC', False], [7, 0, 5, 11, 54, 0.957958, 'DRC', False], [7, -1, 56, 2, 18, 0.718719, 'DRC', False], [7, -2, 41, 12, 25, 0.624124, 'DRC', False], [7, -3, 20, 15, 67, 0.598098, 'DRC', False], [7, -4, 63, 1, 3, 0.522022, 'DRC', False], [7, -5, 20, 2, 76, 0.715215, 'DRC', False], [7, -6, 32, 6, 39, 0.52953, 'DRC', False], [7, 0, 11, 15, 67, 0.525526, 'DRC', False], [7, -1, 19, 4, 64, 0.808308, 'DRC', False], [7, -2, 33, 5, 92, 0.556557, 'DRC', False], [7, -3, 24, 5, 52, 0.657157, 'DRC', False

[32773, 4595, 3485, 279, 31, 0]



/home/jborg/Data/Research/DLPnR/DL-PnR/data/random/logs/nangate45/tinyRocket
	Get data from /home/jborg/Data/Research/DLPnR/DL-PnR/data/random/logs/nangate45/tinyRocket
Start parsing iteration data!
End parsing iteration data!
[[[7, 0, 63, 7, 91, 0.897397, 'ALL', True], [7, -1, 9, 12, 8, 0.563063, 'ALL', True], [7, -2, 3, 4, 69, 0.951451, 'ALL', True], [7, -3, 51, 6, 94, 0.611111, 'DRC', False], [7, -4, 51, 4, 4, 0.876376, 'DRC', False], [7, -5, 20, 8, 73, 0.654154, 'DRC', False], [7, -6, 3, 13, 75, 0.95045, 'DRC', False], [7, 0, 5, 11, 54, 0.957958, 'DRC', False], [7, -1, 56, 2, 18, 0.718719, 'DRC', False], [7, -2, 41, 12, 25, 0.624124, 'DRC', False], [7, -3, 20, 15, 67, 0.598098, 'DRC', False], [7, -4, 63, 1, 3, 0.522022, 'DRC', False], [7, -5, 20, 2, 76, 0.715215, 'DRC', False], [7, -6, 32, 6, 39, 0.52953, 'DRC', False], [7, 0, 11, 15, 67, 0.525526, 'DRC', False], [7, -1, 19, 4, 64, 0.808308, 'DRC', False], [7, -2, 33, 5, 92, 0.556557, 'DRC', False

0